In [ ]:
import os
import csv
import shutil

import numpy as np
import pandas as pd

from matplotlib.lines import Line2D
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as mtick
import matplotlib as mpl
import seaborn as sns

import umap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

from rdkit.Chem import AllChem as Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit.Chem import Lipinski
from rdkit.Chem import Crippen

import warnings
warnings.filterwarnings("ignore")

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams["figure.dpi"] = 300
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']

In [ ]:
root_dir = os.path.join(os.getcwd(), "datasets_int_val")
MUBD_ligand_dir = [os.path.join(it, "MUBDreal/Diverse_ligands_PS.csv") for it in os.scandir(root_dir)]
MUBDreal_decoy_dir = [os.path.join(it, "MUBDreal/Final_decoys.csv") for it in os.scandir(root_dir)]
MUBDsyn_decoy_dir = [os.path.join(it, "MUBDsyn/Final_decoys.csv") for it in os.scandir(root_dir)]

cases = ['5HT1F-AGO', '5HT1F-ANTA', 'DRD5-AGO', 'DRD5-ANTA', 'HRH4-AGO',
        'HRH4-ANTA', 'ACM4-AGO', 'ACM4-ANTA', 'OPRM-AGO', 'OPRM-ANTA',
        'BRS3-ANTA','SSR2-ANTA','AG22-ANTA','PE2R3-AGO','PE2R3-ANTA',
        'MTR1B-AGO','MTR1B-ANTA',
        ]

In [ ]:
#Unique scaffold ratio
def bmsratio(final_decoys):
    df = pd.read_csv(final_decoys)
    smis = list(df['SMILES'])
    mks = []

    for smi in smis:
        mks.append(MurckoScaffoldSmiles(smi))

    mks_u = list(set(mks))
    ratio = len(mks_u) / len(smis)

    return round(ratio, 2)

MUBDreal_usr = []
for case in cases:
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            MUBDreal_usr.append(bmsratio(dir))
MUBDreal_usr = np.array(MUBDreal_usr)

MUBDsyn_usr = []
for case in cases:
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            MUBDsyn_usr.append(bmsratio(dir))
MUBDsyn_usr = np.array(MUBDsyn_usr)

#plot
fig, axes = plt.subplots(1, 1, figsize=(14, 7))
ticks = np.arange(1,18)

plt.axhline(y=1, ls=(0, (5,2.5)), c="black", linewidth=2.5)
sns.lineplot(x=ticks, y=MUBDreal_usr, linestyle='-', color='royalblue', marker='^', linewidth=4, markersize=15, ax=axes)
sns.lineplot(x=ticks, y=MUBDsyn_usr, linestyle='-', color='indianred', marker='s', linewidth=4, markersize=15, ax=axes)
plt.tick_params(labelsize=26)
axes.set_ylabel("Unique scaffold ratio", size="26")
axes.set_xlabel("Case", size="26")
axes.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
axes.set_yticks([0, .20,  .40,  .60,  .80, 1.00])
axes.yaxis.set_major_formatter(plticker.FormatStrFormatter('%.2f'))
axes.grid(True)
axes.legend(["Ideal scaffold diversity", "$\mathdefault{MUBD^{real}}$", "$\mathdefault{MUBD^{syn}}$"])
sns.move_legend(axes, "upper center",bbox_to_anchor=(.5, 1.2), ncol=3, 
                title=None, frameon=False, prop={"size":26})
plt.tight_layout()
fig.savefig('BMSratio.pdf', transparent=True)

In [ ]:
#Property distribution curve
def compute_PDC(diverse_ligand_PS, final_decoys_2, final_decoys_3):
    df_Ligand = diverse_ligand_PS
    df_decoy_2 = final_decoys_2
    df_decoy_3 = final_decoys_3

    def Draw_property(df_Decoy, Pname, length):

        Ligand_P_Max = df_Ligand[Pname].max()
        Ligand_P_Min = df_Ligand[Pname].min()

        Decoy_P_Max = df_Decoy[Pname].max()
        Decoy_P_Min = df_Decoy[Pname].min()

        Ceil = max(int(Ligand_P_Max), int(Decoy_P_Max))
        Flo = min(int(Ligand_P_Min), int(Decoy_P_Min))

        Xstart = (int(Flo / length) - 1) * length
        Xend = (int(Ceil / length) + 2) * length

        total_count_Ligand = len(df_Ligand)
        P_Ligand_count = []
        for i in range(Xstart, Xend, length):
            count = len(df_Ligand[(df_Ligand[Pname] >= i)
                        & (df_Ligand[Pname] < (i + length))])
            pcount = float(count) / float(total_count_Ligand)
            P_Ligand_count.append(pcount)

        total_count_Decoy = len(df_Decoy)
        P_Decoy_count = []
        for i in range(Xstart, Xend, length):
            count = len(df_Decoy[(df_Decoy[Pname] >= i) &
                        (df_Decoy[Pname] < (i + length))])
            pcount = float(count) / float(total_count_Decoy)
            P_Decoy_count.append(pcount)

        X = [Xstart]
        for i in range(Xstart, Xend, length):
            X.append(i + float(length) / 2)
        X.append(Xend)

        P_Ligand_count[0:0] = [0]
        P_Decoy_count[0:0] = [0]

        P_Ligand_count.append(0)
        P_Decoy_count.append(0)

        y_Ligand = P_Ligand_count
        y_Decoy = P_Decoy_count
        x = X

        Draw = [y_Ligand, y_Decoy, x, [Xstart], [Xend]]
        return Draw

    LogP_2 = Draw_property(df_decoy_2,'LogP', 1)
    MW_2 = Draw_property(df_decoy_2,'MW', 100)
    NR_2 = Draw_property(df_decoy_2,'NR', 1)
    NHD_2 = Draw_property(df_decoy_2,'HD', 1)
    NHA_2 = Draw_property(df_decoy_2,'HA', 1)
    FC_2 = Draw_property(df_decoy_2,'FC', 1)

    PDC_files_path_2 = 'PDC_files_2'
    if not os.path.exists(PDC_files_path_2):
        os.makedirs(PDC_files_path_2)

    with open(os.path.join(PDC_files_path_2, "LogP.csv"), 'w', newline='') as csv1:
        w = csv.writer(csv1)
        w.writerows(LogP_2)

    with open(os.path.join(PDC_files_path_2, "MW.csv"), 'w', newline='') as csv2:
        w = csv.writer(csv2)
        w.writerows(MW_2)

    with open(os.path.join(PDC_files_path_2, "NR.csv"), 'w', newline='') as csv3:
        w = csv.writer(csv3)
        w.writerows(NR_2)

    with open(os.path.join(PDC_files_path_2, "NHD.csv"),'w', newline='') as csv4:
        w = csv.writer(csv4)
        w.writerows(NHD_2)

    with open(os.path.join(PDC_files_path_2, "NHA.csv"), 'w', newline='') as csv5:
        w = csv.writer(csv5)
        w.writerows(NHA_2)

    with open(os.path.join(PDC_files_path_2, "FC.csv"),'w', newline='') as csv6:
        w = csv.writer(csv6)
        w.writerows(FC_2)
    LogP_3 = Draw_property(df_decoy_3,'LogP', 1)
    MW_3 = Draw_property(df_decoy_3,'MW', 100)
    NR_3 = Draw_property(df_decoy_3,'NR', 1)
    NHD_3 = Draw_property(df_decoy_3,'HD', 1)
    NHA_3 = Draw_property(df_decoy_3,'HA', 1)
    FC_3 = Draw_property(df_decoy_3,'FC', 1)

    PDC_files_path_3 = 'PDC_files_3'
    if not os.path.exists(PDC_files_path_3):
        os.makedirs(PDC_files_path_3)

    with open(os.path.join(PDC_files_path_3, "LogP.csv"), 'w', newline='') as csv1:
        w = csv.writer(csv1)
        w.writerows(LogP_3)

    with open(os.path.join(PDC_files_path_3, "MW.csv"), 'w', newline='') as csv2:
        w = csv.writer(csv2)
        w.writerows(MW_3)

    with open(os.path.join(PDC_files_path_3, "NR.csv"), 'w', newline='') as csv3:
        w = csv.writer(csv3)
        w.writerows(NR_3)

    with open(os.path.join(PDC_files_path_3, "NHD.csv"),'w', newline='') as csv4:
        w = csv.writer(csv4)
        w.writerows(NHD_3)

    with open(os.path.join(PDC_files_path_3, "NHA.csv"), 'w', newline='') as csv5:
        w = csv.writer(csv5)
        w.writerows(NHA_3)

    with open(os.path.join(PDC_files_path_3, "FC.csv"),'w', newline='') as csv6:
        w = csv.writer(csv6)
        w.writerows(FC_3)


def plot_PDC():

    df_LogP_2 = pd.read_csv('PDC_files_2/LogP.csv', header=None)
    df_LogP_3 = pd.read_csv('PDC_files_3/LogP.csv', header=None)
    LogP_y_Ligand = df_LogP_2.iloc[0]
    LogP_y_Decoy_2 = df_LogP_2.iloc[1]
    LogP_y_Decoy_3 = df_LogP_3.iloc[1]
    LogP_x = df_LogP_2.iloc[2]
    LogP_Xstart = df_LogP_2.iloc[3, 0]
    LogP_Xend = df_LogP_2.iloc[4, 0]

    df_MW_2 = pd.read_csv('PDC_files_2/MW.csv', header=None)
    df_MW_3 = pd.read_csv('PDC_files_3/MW.csv', header=None)
    MW_y_Ligand = df_MW_2.iloc[0]
    MW_y_Decoy_2 = df_MW_2.iloc[1]
    MW_y_Decoy_3 = df_MW_3.iloc[1]
    MW_x = df_MW_2.iloc[2]
    MW_Xstart = df_MW_2.iloc[3, 0]
    MW_Xend = df_MW_2.iloc[4, 0]

    df_NR_2 = pd.read_csv('PDC_files_2/NR.csv', header=None)
    df_NR_3 = pd.read_csv('PDC_files_3/NR.csv', header=None)
    NR_y_Ligand = df_NR_2.iloc[0]
    NR_y_Decoy_2 = df_NR_2.iloc[1]
    NR_y_Decoy_3 = df_NR_3.iloc[1]
    NR_x = df_NR_2.iloc[2]
    NR_Xstart = df_NR_2.iloc[3, 0]
    NR_Xend = df_NR_2.iloc[4, 0]

    df_NHD_2 = pd.read_csv('PDC_files_2/NHD.csv', header=None)
    df_NHD_3 = pd.read_csv('PDC_files_3/NHD.csv', header=None)
    NHD_y_Ligand = df_NHD_2.iloc[0]
    NHD_y_Decoy_2 = df_NHD_2.iloc[1]
    NHD_y_Decoy_3 = df_NHD_3.iloc[1]
    NHD_x = df_NHD_2.iloc[2]
    NHD_Xstart = df_NHD_2.iloc[3, 0]
    NHD_Xend = df_NHD_2.iloc[4, 0]

    df_NHA_2 = pd.read_csv('PDC_files_2/NHA.csv', header=None)
    df_NHA_3 = pd.read_csv('PDC_files_3/NHA.csv', header=None)
    NHA_y_Ligand = df_NHA_2.iloc[0]
    NHA_y_Decoy_2 = df_NHA_2.iloc[1]
    NHA_y_Decoy_3 = df_NHA_3.iloc[1]
    NHA_x = df_NHA_2.iloc[2]
    NHA_Xstart = df_NHA_2.iloc[3, 0]
    NHA_Xend = df_NHA_2.iloc[4, 0]

    df_FC_2 = pd.read_csv('PDC_files_2/FC.csv', header=None)
    df_FC_3 = pd.read_csv('PDC_files_3/FC.csv', header=None)
    FC_y_Ligand = df_FC_2.iloc[0]
    FC_y_Decoy_2 = df_FC_2.iloc[1]
    FC_y_Decoy_3 = df_FC_3.iloc[1]
    FC_x = df_FC_2.iloc[2]
    FC_Xstart = df_FC_2.iloc[3, 0]
    FC_Xend = df_FC_2.iloc[4, 0]

    fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(14, 7))

    ax1.set_xlabel('LogP', size=26)
    ax1.set_ylabel('Fraction', size=26)
    ax1.set_xlim(LogP_Xstart, LogP_Xend)
    ax1.set_ylim(0, 1)
    sns.lineplot(x=LogP_x, y=LogP_y_Ligand, color="forestgreen", lw=4, ax=ax1)
    sns.lineplot(x=LogP_x, y=LogP_y_Decoy_2, color="royalblue", lw=4, ax=ax1)
    sns.lineplot(x=LogP_x, y=LogP_y_Decoy_3, color="indianred", lw=4, ax=ax1)
    ax1.set_xticks([-2.0, 1.0,  4.0,  7.0, 10.0])
    ax1.set_yticks([0, .2,  .4,  .6,  .8, 1.0])
    ax1.tick_params(labelsize=26)

    ax2.set_xlabel('Molecular weight (Da)', size=26)
    ax2.set_ylabel(' ')
    ax2.set_xlim(MW_Xstart, MW_Xend)
    ax2.set_ylim(0, 1)
    sns.lineplot(x=MW_x, y=MW_y_Ligand, color = "forestgreen", label="MUBD ligands", lw=4, ax=ax2)
    sns.lineplot(x=MW_x, y=MW_y_Decoy_2, color="royalblue", label="$\mathdefault{MUBD^{real}}$ decoys", lw=4, ax=ax2)
    sns.lineplot(x=MW_x, y=MW_y_Decoy_3, color="indianred", label="$\mathdefault{MUBD^{syn}}$ decoys", lw=4, ax=ax2)
    ax2.set_xticks([0, 200,  400,  600, 800])
    ax2.set_yticks([0, .2,  .4,  .6,  .8, 1.0])
    ax2.get_legend().remove() 
    ax2.tick_params(labelsize=26)

    ax3.set_xlabel('Rotatable bonds', size=26)
    ax3.set_ylabel(' ')
    ax3.set_xlim(NR_Xstart, NR_Xend)
    ax3.set_ylim(0, 1)
    sns.lineplot(x=NR_x, y=NR_y_Ligand, color = "forestgreen", lw=4, ax=ax3)
    sns.lineplot(x=NR_x, y=NR_y_Decoy_2, color="royalblue", lw=4, ax=ax3)
    sns.lineplot(x=NR_x, y=NR_y_Decoy_3, color="indianred", lw=4, ax=ax3)   
    ax3.set_xticks([0, 5, 10, 15])
    ax3.set_yticks([0, .2, .4, .6, .8, 1.0])
    ax3.tick_params(labelsize=26)

    ax4.set_xlabel('Hydrogen bond donors', size=26)
    ax4.set_ylabel('Fraction', size=26)
    ax4.set_xlim(NHD_Xstart, NHD_Xend)
    ax4.set_ylim(0, 1)
    sns.lineplot(x=NHD_x, y=NHD_y_Ligand, color = "forestgreen", lw=4, ax=ax4)
    sns.lineplot(x=NHD_x, y=NHD_y_Decoy_2, color="royalblue", lw=4, ax=ax4)
    sns.lineplot(x=NHD_x, y=NHD_y_Decoy_3, color="indianred", lw=4, ax=ax4)
    ax4.set_xticks([0, 2, 4, 6, 8])
    ax4.set_yticks([0, .2,  .4,  .6,  .8, 1.0])    
    ax4.tick_params(labelsize=26)

    ax5.set_xlabel('Hydrogen bond acceptors', size=26)
    ax5.set_ylabel(' ')
    ax5.set_xlim(NHA_Xstart, NHA_Xend)
    ax5.set_ylim(0, 1)
    sns.lineplot(x=NHA_x, y=NHA_y_Ligand, color = "forestgreen", lw=4, ax=ax5)
    sns.lineplot(x=NHA_x, y=NHA_y_Decoy_2, color="royalblue", lw=4, ax=ax5)
    sns.lineplot(x=NHA_x, y=NHA_y_Decoy_3, color="indianred", lw=4, ax=ax5)
    ax5.set_xticks([0, 2, 4, 6, 8, 10])
    ax5.set_yticks([0, .2, .4, .6, .8, 1.0])        
    ax5.tick_params(labelsize=26)

    ax6.set_xlabel('Net charge', size=26)
    ax6.set_ylabel(' ')
    ax6.set_xlim(FC_Xstart, FC_Xend)
    ax6.set_ylim(0, 1)
    sns.lineplot(x=FC_x, y=FC_y_Ligand, color = "forestgreen", lw=4, ax=ax6)
    sns.lineplot(x=FC_x, y=FC_y_Decoy_2, color="royalblue", lw=4, ax=ax6)
    sns.lineplot(x=FC_x, y=FC_y_Decoy_3, color="indianred", lw=4, ax=ax6)
    ax6.set_xticks([-2, 0, 2, 4])
    ax6.set_yticks([0, .2, .4, .6, .8, 1.0])      
    ax6.tick_params(labelsize=26)

    plt.figlegend()
    sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 1.1), ncol=3, 
                title=None, frameon=False, prop={"size":26})
    plt.tight_layout()
    fig.savefig("pdc.pdf", transparent=True, bbox_inches = 'tight')
    
    shutil.rmtree("PDC_files_2")
    shutil.rmtree("PDC_files_3")

df_ligand = [pd.read_csv(n) for n in MUBD_ligand_dir]
df_ligand = pd.concat(df_ligand, axis=0)

df_decoy_real = [pd.read_csv(n) for n in MUBDreal_decoy_dir]
df_decoy_real = pd.concat(df_decoy_real, axis=0)

df_decoy_syn = [pd.read_csv(n) for n in MUBDsyn_decoy_dir]
df_decoy_syn = pd.concat(df_decoy_syn, axis=0)

compute_PDC(df_ligand, df_decoy_real, df_decoy_syn)
plot_PDC()

In [ ]:
#simp
def simp_AUC(idx, diverse_ligand_PS, final_decoys):
    df_Ligand = pd.read_csv(diverse_ligand_PS)
    df_Decoy = pd.read_csv(final_decoys)
    length_Ligand = len(df_Ligand)
    length_Decoy = len(df_Decoy)

    MW_MCS_Ligand = list(df_Ligand['MW_MCS'])
    NR_MCS_Ligand = list(df_Ligand['NR_MCS'])
    HD_MCS_Ligand = list(df_Ligand['HD_MCS'])
    HA_MCS_Ligand = list(df_Ligand['HA_MCS'])
    FC_MCS_Ligand = list(df_Ligand['FC_MCS'])
    LogP_MCS_Ligand = list(df_Ligand['LogP_MCS'])

    MW_MCS_Decoy = list(df_Decoy['MW_MCS'])
    NR_MCS_Decoy = list(df_Decoy['NR_MCS'])
    HD_MCS_Decoy = list(df_Decoy['HD_MCS'])
    HA_MCS_Decoy = list(df_Decoy['HA_MCS'])
    FC_MCS_Decoy = list(df_Decoy['FC_MCS'])
    LogP_MCS_Decoy = list(df_Decoy['LogP_MCS'])

    def Middle(PIT, PIR):
        x = PIT - PIR
        y = np.square(x)
        return(y)

    def simp_LL(t, r):
        simp1 = Middle(MW_MCS_Ligand[t], MW_MCS_Ligand[r])
        simp2 = Middle(NR_MCS_Ligand[t], NR_MCS_Ligand[r])
        simp3 = Middle(HD_MCS_Ligand[t], HD_MCS_Ligand[r])
        simp4 = Middle(HA_MCS_Ligand[t], HA_MCS_Ligand[r])
        simp5 = Middle(FC_MCS_Ligand[t], FC_MCS_Ligand[r])
        simp6 = Middle(LogP_MCS_Ligand[t], LogP_MCS_Ligand[r])
        simp = 1 - np.sqrt((simp1 + simp2 + simp3 +
                              simp4 + simp5 + simp6) / 6)
        return simp

    def simp_LD(t, r):
        simp1 = Middle(MW_MCS_Ligand[t], MW_MCS_Decoy[r])
        simp2 = Middle(NR_MCS_Ligand[t], NR_MCS_Decoy[r])
        simp3 = Middle(HD_MCS_Ligand[t], HD_MCS_Decoy[r])
        simp4 = Middle(HA_MCS_Ligand[t], HA_MCS_Decoy[r])
        simp5 = Middle(FC_MCS_Ligand[t], FC_MCS_Decoy[r])
        simp6 = Middle(LogP_MCS_Ligand[t], LogP_MCS_Decoy[r])
        simp = 1 - np.sqrt((simp1 + simp2 + simp3 +
                              simp4 + simp5 + simp6) / 6)
        return simp

    def Get_simp(k):
        Simp = []

        for i in range(0, length_Ligand):
            if i != k:
                Simp.append(1)
                Simp.append(simp_LL(k, i))

        for j in range(0, length_Decoy):
            if not ((39 * k <= j) & (j < 39 * (k + 1))):
                Simp.append(0)
                Simp.append(simp_LD(k, j))
        return Simp

    def Get_Roc_Arg_simp(k):
        Simp_list = Get_simp(k)
        data = np.array(Simp_list).reshape(-1, 2)
        df_data = pd.DataFrame(data, columns=['sort', 'simp'])
        df = df_data.sort_values(by="simp", ascending=False)
        sort = list(df['sort'])
        simp = list(df['simp'])

        fpr, tpr, thresholds = roc_curve(sort, simp)
        roc_auc = auc(fpr, tpr)

        return (fpr, tpr, roc_auc)

    Fpr_list_simp = []
    Tpr_list_simp = []
    Auc_list_simp = []

    for k in range(0, length_Ligand):
        Roc_Arg = Get_Roc_Arg_simp(k)
        Fpr_list_simp.append(Roc_Arg[0])
        Tpr_list_simp.append(Roc_Arg[1])
        Auc_list_simp.append(Roc_Arg[2])

    auc_simps = np.array(Auc_list_simp)
    idx_strs = [str(idx+1)] * len(auc_simps)
    df = pd.DataFrame({"case":idx_strs, "simp":auc_simps})

    return df

MUBDreal_dfs, MUBDsyn_dfs = [], []
for i, case in enumerate(cases):
    for dir in MUBD_ligand_dir:
        if case in dir:
            single_ligands = dir
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            single_real_decoys = dir
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            single_syn_decoys = dir
    
    df_real = simp_AUC(i, single_ligands, single_real_decoys)
    MUBDreal_dfs.append(df_real)

    df_syn = simp_AUC(i, single_ligands, single_syn_decoys)
    MUBDsyn_dfs.append(df_syn)

MUBDreal_simp = pd.concat(MUBDreal_dfs).reset_index()
MUBDsyn_simp = pd.concat(MUBDsyn_dfs).reset_index()

fig, axes = plt.subplots(1, 1, figsize=(14, 7))
plt.axhline(y=0.5, ls=(0, (5,2.5)), c="black", linewidth=2.5)
sns.lineplot(x="case", y="simp", data=MUBDreal_simp, color='royalblue', 
        err_style="bars", err_kws={'capsize':6, 'elinewidth':2, 'capthick':2}, marker='^', linewidth=4, markersize=15, ax=axes)
sns.lineplot(x="case", y="simp", data=MUBDsyn_simp, color='indianred', 
        err_style="bars", err_kws={'capsize':6, 'elinewidth':2, 'capthick':2}, marker='s', linewidth=4, markersize=15, ax=axes)

plt.tick_params(labelsize=26)

axes.yaxis.set_minor_locator(MultipleLocator(0.1))
axes.set_ylabel("mean(AUCs)",  size=26)
axes.set_xlabel("Case",  size=26)

axes.set_yticks([0.0, 0.25,0.50, 0.75,1.00])

axes.grid(True)
plt.tight_layout()
axes.legend(["Random distribution", "$\mathdefault{MUBD^{real}}$", "$\mathdefault{MUBD^{syn}}$"])
sns.move_legend(axes, "upper center",bbox_to_anchor=(.5, 1.2), ncol=3, 
                title=None, frameon=False, prop={"size":26})
fig.savefig("simp.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#NLBscore
def nlbscore(diverse_ligands_PS, final_decoys):
    df_Ligand = pd.read_csv(diverse_ligands_PS)
    df_Decoy = pd.read_csv(final_decoys)
    length_Ligand = len(df_Ligand)
    length_Decoy = len(df_Decoy)

    S_Ligand = list(df_Ligand['SMILES'])
    Suppl_Ligand = []
    for i in range(0, length_Ligand):
        m = Chem.MolFromSmiles(S_Ligand[i])
        Suppl_Ligand.append(m)

    S_Decoy = list(df_Decoy['SMILES'])
    Suppl_Decoy = []
    for i in range(0, length_Decoy):
        m = Chem.MolFromSmiles(S_Decoy[i])
        Suppl_Decoy.append(m)

    Fps_Ligand = [MACCSkeys.GenMACCSKeys(x) for x in Suppl_Ligand]
    Fps_Decoy = [MACCSkeys.GenMACCSKeys(x) for x in Suppl_Decoy]

    def sims_LL(t, r):
        sims = DataStructs.FingerprintSimilarity(Fps_Ligand[t], Fps_Ligand[r])
        return sims

    def sims_LD(t, r):
        sims = DataStructs.FingerprintSimilarity(Fps_Ligand[t], Fps_Decoy[r])
        return sims

    def Get_NLB(k):
        Sims_LD_list = []
        for i in range(0, length_Decoy):
            Sims_LD_list.append(sims_LD(k, i))

        Sims_LD_Max = max(Sims_LD_list[i] for i in range(0, len(Sims_LD_list)))

        Sims_LL_list = []
        for i in range(0, length_Ligand):
            if i != k:
                Sims_LL_list.append(sims_LL(k, i))

        count = 0
        for i in range(0, len(Sims_LL_list)):
            Sims_LL = Sims_LL_list[i]
            if Sims_LL > Sims_LD_Max:
                count = count + 1
        Pcount = float(count) / float(len(Sims_LL_list))

        return Pcount

    NLB_Sum = 0
    for k in range(0, length_Ligand):
        Pcount = Get_NLB(k)
        NLB_Sum = NLB_Sum + Pcount
    score = float(NLB_Sum) / float(length_Ligand)

    return round(score, 3)

MUBDreal_nlbs, MUBDsyn_nlbs = [], []
for i, case in enumerate(cases):
    for dir in MUBD_ligand_dir:
        if case in dir:
            single_ligands = dir
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            single_real_decoys = dir
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            single_syn_decoys = dir
    
    nlb_real = nlbscore(single_ligands, single_real_decoys)
    MUBDreal_nlbs.append(nlb_real)

    nlb_syn = nlbscore(single_ligands, single_syn_decoys)
    MUBDsyn_nlbs.append(nlb_syn)

MUBDreal_nlbs = np.array(MUBDreal_nlbs)
MUBDsyn_nlbs = np.array(MUBDsyn_nlbs)

fig, axes = plt.subplots(1, 1, figsize=(14, 7))

x = np.arange(17)
tick_labels = np.arange(1,18)

bar_width = 0.32

axes.grid(axis="y")
axes.set_axisbelow(True)
line1 = plt.axhline(y=0, ls=(0, (5,2.5)), c="black", linewidth=2.5)
bar1 = plt.bar(x, MUBDreal_nlbs, bar_width, align="center", color='royalblue', tick_label=tick_labels)
bar2 = plt.bar(x+bar_width, MUBDsyn_nlbs, bar_width, align="center", color='indianred', tick_label=tick_labels)
plt.xticks(x+0.5*bar_width, tick_labels)

axes.tick_params(labelsize=26)
axes.set_ylabel("NLBscore",  size=26)
axes.set_xlabel("Case",  size=26)

axes.set_yticks(np.arange(0, 0.22, 0.04))
axes.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
axes.legend(["Free of NL bias", "$\mathdefault{MUBD^{real}}$", "$\mathdefault{MUBD^{syn}}$"])
sns.move_legend(axes, "upper center",bbox_to_anchor=(.5, 1.2), ncol=3, 
                title=None, frameon=False, prop={"size":26})
plt.tight_layout()
fig.savefig("nlbscore.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#sims
def sims_AUC(idx, diverse_ligand_PS, final_decoys):

    df_Ligand = pd.read_csv(diverse_ligand_PS)
    df_Decoy = pd.read_csv(final_decoys)
    length_Ligand = len(df_Ligand)
    length_Decoy = len(df_Decoy)

    S_Ligand = list(df_Ligand['SMILES'])
    Suppl_Ligand = []
    for i in range(0, length_Ligand):
        m = Chem.MolFromSmiles(S_Ligand[i])
        Suppl_Ligand.append(m)

    S_Decoy = list(df_Decoy['SMILES'])
    Suppl_Decoy = []
    for i in range(0, length_Decoy):
        m = Chem.MolFromSmiles(S_Decoy[i])
        Suppl_Decoy.append(m)

    Fps_Ligand = [MACCSkeys.GenMACCSKeys(x) for x in Suppl_Ligand]
    Fps_Decoy = [MACCSkeys.GenMACCSKeys(x) for x in Suppl_Decoy]

    def sims_LL(t, r):
        sims = DataStructs.FingerprintSimilarity(Fps_Ligand[t], Fps_Ligand[r])
        return sims

    def sims_LD(t, r):
        sims = DataStructs.FingerprintSimilarity(Fps_Ligand[t], Fps_Decoy[r])
        return sims

    def Get_sims(k):
        Sims = []

        for i in range(0, length_Ligand):
            if i != k:
                Sims.append(1)
                Sims.append(sims_LL(k, i))
        for j in range(0, length_Decoy):
            if not ((39 * k <= j) & (j < 39 * (k + 1))):
                Sims.append(0)
                Sims.append(sims_LD(k, j))
        return Sims

    def Get_Roc_Arg_sims(k):
        Sims_list = Get_sims(k)
        data = np.array(Sims_list).reshape(-1, 2)
        df_data = pd.DataFrame(data, columns=['sort', 'sims'])
        df = df_data.sort_values(by="sims", ascending=False)
        sort = list(df['sort'])
        sims = list(df['sims'])

        fpr, tpr, thresholds = roc_curve(sort, sims)
        roc_auc = auc(fpr, tpr)

        return (fpr, tpr, roc_auc)

    Fpr_list_sims = []
    Tpr_list_sims = []
    Auc_list_sims = []

    for k in range(0, length_Ligand):
        Roc_Arg = Get_Roc_Arg_sims(k)
        Fpr_list_sims.append(Roc_Arg[0])
        Tpr_list_sims.append(Roc_Arg[1])
        Auc_list_sims.append(Roc_Arg[2])

    auc_simss = np.array(Auc_list_sims)
    idx_strs = [str(idx+1)] * len(auc_simss)
    df = pd.DataFrame({"case":idx_strs, "sims":auc_simss})

    return df

MUBDreal_dfs, MUBDsyn_dfs = [], []
for i, case in enumerate(cases):
    for dir in MUBD_ligand_dir:
        if case in dir:
            single_ligands = dir
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            single_real_decoys = dir
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            single_syn_decoys = dir
    
    df_real = sims_AUC(i, single_ligands, single_real_decoys)
    MUBDreal_dfs.append(df_real)

    df_syn = sims_AUC(i, single_ligands, single_syn_decoys)
    MUBDsyn_dfs.append(df_syn)

MUBDreal_sims = pd.concat(MUBDreal_dfs).reset_index()
MUBDsyn_sims = pd.concat(MUBDsyn_dfs).reset_index()

fig, axes = plt.subplots(1, 1, figsize=(14, 7))
plt.axhline(y=0.5, ls=(0, (5,2.5)), c="black", linewidth=2.5)
sns.lineplot(x="case", y="sims", data=MUBDreal_sims, color='royalblue', 
        err_style="bars", err_kws={'capsize':6, 'elinewidth':2, 'capthick':2}, marker='^', linewidth=4, markersize=15, ax=axes)
sns.lineplot(x="case", y="sims", data=MUBDsyn_sims, color='indianred', 
        err_style="bars", err_kws={'capsize':6, 'elinewidth':2, 'capthick':2}, marker='s', linewidth=4, markersize=15, ax=axes)

plt.tick_params(labelsize=26)

axes.yaxis.set_minor_locator(MultipleLocator(0.1))
axes.set_ylabel("mean(AUCs)",  size=26)
axes.set_xlabel("Case",  size=26)

axes.set_yticks([0.0, 0.25,0.50, 0.75,1.00])

axes.grid(True)
plt.tight_layout()
axes.legend(["Random distribution", "$\mathdefault{MUBD^{real}}$", "$\mathdefault{MUBD^{syn}}$"])
sns.move_legend(axes, "upper center",bbox_to_anchor=(.5, 1.2), ncol=3, 
                title=None, frameon=False, prop={"size":26})
fig.savefig("sims.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#UMAP
def read_mols(df):
    smis = list(df["SMILES"])
    mols = [Chem.MolFromSmiles(smi) for smi in smis]
    return mols

def get_fps(mols):
    fps = []
    for mol in mols:
        bv = Chem.GetMACCSKeysFingerprint(mol)
        fp = np.zeros(len(bv))
        DataStructs.ConvertToNumpyArray(bv, fp)
        fps.append(fp)
    return np.array(fps)

def get_prop(mols):
    props = []
    for mol in mols:
        mw = Descriptors.MolWt(mol)
        nr = Lipinski.NumRotatableBonds(mol)
        hd = Lipinski.NumHDonors(mol)
        ha = Lipinski.NumHAcceptors(mol)
        fc = rdmolops.GetFormalCharge(mol)
        LogP = Crippen.MolLogP(mol)
        prop = np.array([mw, nr, hd, ha, fc, LogP])
        props.append(prop)
    return np.array(props)

def embedding_umap(data):
    data_scaled = StandardScaler().fit_transform(data)
    reducer = umap.UMAP(n_neighbors=40,
                        min_dist=0.85,
                        metric='correlation',
                        random_state=3)
    embeds = reducer.fit_transform(data_scaled)
    return embeds

df_ligand = [pd.read_csv(n) for n in MUBD_ligand_dir]
df_ligand = pd.concat(df_ligand, axis=0)

df_decoy_real = [pd.read_csv(n) for n in MUBDreal_decoy_dir]
df_decoy_real = pd.concat(df_decoy_real, axis=0)

df_decoy_syn = [pd.read_csv(n) for n in MUBDsyn_decoy_dir]
df_decoy_syn = pd.concat(df_decoy_syn, axis=0)

ligands = read_mols(df_ligand)
decoys_2 = read_mols(df_decoy_real)
decoys_3 = read_mols(df_decoy_syn)

l_2_fps = get_fps(ligands)
d_2_fps = get_fps(decoys_2)
d_3_fps = get_fps(decoys_3)
fps_all = np.concatenate((l_2_fps, d_2_fps, d_3_fps), axis=0)

fps_embeds = embedding_umap(fps_all)

l_2_props = get_prop(ligands)
d_2_props = get_prop(decoys_2)
d_3_props = get_prop(decoys_3)
props_all = np.concatenate((l_2_props, d_2_props, d_3_props), axis=0)

props_embeds = embedding_umap(props_all)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14, 7))
legend_elements = [Line2D([0], [0], marker='o', color='r', markeredgecolor="black",
                          markerfacecolor='lime', markersize=4),
                Line2D([0], [0], marker='o', color='r', 
                          markerfacecolor='royalblue', markersize=2.5),
                Line2D([0], [0], marker='o', color='r',
                          markerfacecolor='indianred', markersize=2.5),]
labels = ['MUBD ligands', '$\mathdefault{MUBD^{real}}$ decoys', '$\mathdefault{MUBD^{syn}}$ decoys']   

s_1d_real = ax1.scatter(props_embeds[len(ligands):(len(ligands)+len(decoys_2)), 0], props_embeds[len(ligands):(len(ligands)+len(decoys_2)), 1],
        c="royalblue", s=8,edgecolors="black", linewidths=0,alpha=0.8)
s_1d_syn = ax1.scatter(props_embeds[(len(ligands)+len(decoys_2)):, 0], props_embeds[(len(ligands)+len(decoys_2)):, 1],
        c="indianred", s=8,edgecolors="black", linewidths=0,alpha=0.8)
s_1d_ligands = ax1.scatter(props_embeds[:len(ligands), 0], props_embeds[:len(ligands), 1],
        c="lime", s=32, marker="o", edgecolors="black", linewidths=1)
ax1.set_xlim([np.min(props_embeds[:,0])-0.5, np.max(props_embeds[:,0])+1.5])
ax1.set_ylim([np.min(props_embeds[:,1])-0.5, np.max(props_embeds[:,1])+0.5])
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax1.get_yticklabels(), visible=False)
ax1.tick_params(axis='both', which='both',length=0)
ax1.set_xlabel('UMAP 1',size=26)
ax1.set_ylabel("UMAP 2",size=26)
ax1.set_title("Physicochemical descriptors",size=26)


s_2d_real = ax2.scatter(fps_embeds[len(ligands):(len(ligands)+len(decoys_2)), 0], fps_embeds[len(ligands):(len(ligands)+len(decoys_2)), 1],
        c="royalblue", s=8, linewidths=0,alpha=0.8)
s_2d_syn = ax2.scatter(fps_embeds[(len(ligands)+len(decoys_2)):, 0], fps_embeds[(len(ligands)+len(decoys_2)):, 1],
        c="indianred", s=8, linewidths=0,alpha=0.8)
s_2d_ligands = ax2.scatter(fps_embeds[:len(ligands), 0], fps_embeds[:len(ligands), 1],
        c="lime", s=32, marker="o", edgecolors="black", linewidths=1,)
ax2.set_xlim([np.min(fps_embeds[:,0])-0.5, np.max(fps_embeds[:,0])+1.5])
ax2.set_ylim([np.min(fps_embeds[:,1])-0.5, np.max(fps_embeds[:,1])+0.5])
ax2.tick_params(axis='both', which='both',length=0)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
ax2.set_xlabel('UMAP 1',size=26)
ax2.set_ylabel("UMAP 2",size=26)
ax2.set_title("MACCS keys",size=26)

fig.legend((s_2d_ligands, s_2d_real, s_2d_syn), labels)
sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 1.05), ncol=3, 
                title=None, frameon=False, prop={"size":26}, markerscale=3)
box = ax1.get_position()
ax1.set_position([box.x0, box.y0, box.width, box.height*0.9])
box = ax2.get_position()
ax2.set_position([box.x0, box.y0, box.width, box.height*0.9])
fig.savefig("UMAP.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#UMAP SIa
def read_mols(path):
    df = pd.read_csv(path)
    smis = list(df["SMILES"])
    mols = [Chem.MolFromSmiles(smi) for smi in smis]
    return mols

def get_props(mols):
    props = []
    for mol in mols:
        mw = Descriptors.MolWt(mol)
        nr = Lipinski.NumRotatableBonds(mol)
        hd = Lipinski.NumHDonors(mol)
        ha = Lipinski.NumHAcceptors(mol)
        fc = rdmolops.GetFormalCharge(mol)
        LogP = Crippen.MolLogP(mol)
        prop = np.array([mw, nr, hd, ha, fc, LogP])
        props.append(prop)
    return np.array(props)

def embedding_umap(data):
    data_scaled = StandardScaler().fit_transform(data)
    reducer = umap.UMAP(n_neighbors=20,
                        min_dist=0.5,
                        metric='correlation',
                        random_state=3)
    embeds = reducer.fit_transform(data_scaled)
    return embeds

idx = 0
fig, axes = plt.subplots(5,4, figsize=(28, 35))
labels = ['MUBD ligands', '$\mathdefault{MUBD^{real}}$ decoys', '$\mathdefault{MUBD^{syn}}$ decoys',]
for p, case in enumerate(cases):
    for dir in MUBD_ligand_dir:
        if case in dir:
            single_ligands = dir
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            single_real_decoys = dir
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            single_syn_decoys = dir
    ligand = read_mols(single_ligands)
    ligand_props = get_props(ligand)
    decoy_real = read_mols(single_real_decoys)
    decoy_real_props = get_props(decoy_real)
    decoy_syn = read_mols(single_syn_decoys)
    decoy_syn_props = get_props(decoy_syn)
    props_all = np.concatenate((ligand_props, decoy_real_props, decoy_syn_props), axis=0)
    props_embeds = embedding_umap(props_all)
    i, j = int(p / 4), int(p % 4)
    s_1d_real = axes[i,j].scatter(props_embeds[len(ligand):(len(ligand)+len(decoy_real)), 0], props_embeds[len(ligand):(len(ligand)+len(decoy_real)), 1],
            c="royalblue", s=12, linewidths=0, alpha=0.8)
    s_1d_syn = axes[i,j].scatter(props_embeds[(len(ligand)+len(decoy_real)):, 0], props_embeds[(len(ligand)+len(decoy_real)):, 1],
            c="indianred", s=12, linewidths=0,alpha=0.8)
    s_1d_ligands = axes[i,j].scatter(props_embeds[:len(ligand), 0], props_embeds[:len(ligand), 1],
            c="lime", s=48, marker="o", edgecolors="black", linewidths=1)
    axes[i,j].set_xlim([np.min(props_embeds[:,0])-0.5, np.max(props_embeds[:,0])+1.5])
    axes[i,j].set_ylim([np.min(props_embeds[:,1])-0.5, np.max(props_embeds[:,1])+0.5])
    axes[i,j].tick_params(axis='both', which='both',length=0)
    plt.setp(axes[i,j].get_xticklabels(), visible=False)
    plt.setp(axes[i,j].get_yticklabels(), visible=False)
    axes[i,j].set_xlabel('UMAP 1',size=26)
    axes[i,j].set_ylabel("UMAP 2",size=26)
    axes[i,j].set_title(case, size=26)

for k in range(20):
    i, j = int(k / 4), int(k % 4)
    if j == 0 and i != 4:
        axes[i,j].set(xlabel=None)
    if i == 3 and j != 0:
        axes[i,j].set(ylabel=None)
    if j != 0 and i != 3 and i != 4:
        axes[i,j].set(xlabel=None, ylabel=None)
    if i == 4 and j != 0:
        axes[i,j].axis("off")
        
fig.legend((s_1d_ligands, s_1d_real, s_1d_syn), labels)
sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 0.93), ncol=3, 
                title=None, frameon=False, prop={"size":26}, markerscale=3)
fig.savefig("UMAP_SIa.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#UMAP SIb
def read_mols(path):
    df = pd.read_csv(path)
    smis = list(df["SMILES"])
    mols = [Chem.MolFromSmiles(smi) for smi in smis]
    return mols

def get_fps(mols):
    fps = []
    for mol in mols:
        bv = Chem.GetMACCSKeysFingerprint(mol)
        fp = np.zeros(len(bv))
        DataStructs.ConvertToNumpyArray(bv, fp)
        fps.append(fp)
    return np.array(fps)

def embedding_umap(data):
    data_scaled = StandardScaler().fit_transform(data)
    reducer = umap.UMAP(n_neighbors=10,
                        min_dist=0.25,
                        metric='correlation',
                        random_state=3)
    embeds = reducer.fit_transform(data_scaled)
    return embeds

idx = 0
fig, axes = plt.subplots(5,4, figsize=(28, 35))
labels = ['MUBD ligands', '$\mathdefault{MUBD^{real}}$ decoys', '$\mathdefault{MUBD^{syn}}$ decoys',]
for p, case in enumerate(cases):
    for dir in MUBD_ligand_dir:
        if case in dir:
            single_ligands = dir
    for dir in MUBDreal_decoy_dir:
        if case in dir:
            single_real_decoys = dir
    for dir in MUBDsyn_decoy_dir:
        if case in dir:
            single_syn_decoys = dir
    ligand = read_mols(single_ligands)
    ligand_fps = get_fps(ligand)
    decoy_real = read_mols(single_real_decoys)
    decoy_real_fps = get_fps(decoy_real)
    decoy_syn = read_mols(single_syn_decoys)
    decoy_syn_fps = get_fps(decoy_syn)
    fps_all = np.concatenate((ligand_fps, decoy_real_fps, decoy_syn_fps), axis=0)
    fps_embeds = embedding_umap(fps_all)
    i, j = int(p / 4), int(p % 4)
    s_2d_real = axes[i,j].scatter(fps_embeds[len(ligand):(len(ligand)+len(decoy_real)), 0], fps_embeds[len(ligand):(len(ligand)+len(decoy_real)), 1],
            c="royalblue", s=12, linewidths=0, alpha=0.8)
    s_2d_syn = axes[i,j].scatter(fps_embeds[(len(ligand)+len(decoy_real)):, 0], fps_embeds[(len(ligand)+len(decoy_real)):, 1],
            c="indianred", s=12, linewidths=0,alpha=0.8)
    s_2d_ligands = axes[i,j].scatter(fps_embeds[:len(ligand), 0], fps_embeds[:len(ligand), 1],
            c="lime", s=48, marker="o", edgecolors="black", linewidths=1)
    axes[i,j].set_xlim([np.min(fps_embeds[:,0])-0.5, np.max(fps_embeds[:,0])+1.5])
    axes[i,j].set_ylim([np.min(fps_embeds[:,1])-0.5, np.max(fps_embeds[:,1])+0.5])
    axes[i,j].tick_params(axis='both', which='both',length=0)
    plt.setp(axes[i,j].get_xticklabels(), visible=False)
    plt.setp(axes[i,j].get_yticklabels(), visible=False)
    axes[i,j].set_xlabel('UMAP 1',size=26)
    axes[i,j].set_ylabel("UMAP 2",size=26)
    axes[i,j].set_title(case, size=26)

for k in range(20):
    i, j = int(k / 4), int(k % 4)
    if j == 0 and i != 4:
        axes[i,j].set(xlabel=None)
    if i == 3 and j != 0:
        axes[i,j].set(ylabel=None)
    if j != 0 and i != 3 and i != 4:
        axes[i,j].set(xlabel=None, ylabel=None)
    if i == 4 and j != 0:
        axes[i,j].axis("off")
        
fig.legend((s_2d_ligands, s_2d_real, s_2d_syn), labels)
sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 0.93), ncol=3, 
                title=None, frameon=False, prop={"size":26}, markerscale=3)
fig.savefig("UMAP_SIb.pdf", transparent=True, bbox_inches='tight')